In [ ]:
import cv2
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
def drawBox(img,bbox):
    x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
    cv2.rectangle(img, (x, y), ((x + w), (y + h)), (255, 0, 255), 3, 3 )
    cv2.putText(img, "Tracking", (100, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

myPath = 'data/images'
cameraNo = 0
cameraBrightness = 180
moduleVal = 10  # SAVE EVERY ITH FRAME TO AVOID REPETITION
minBlur = 500  # SMALLER VALUE MEANS MORE BLURRINESS PRESENT
grayImage = False # IMAGES SAVED COLORED OR GRAY
saveData = True   # SAVE DATA FLAG
showImage = True  # IMAGE DISPLAY FLAG
imgWidth = 224
imgHeight = 224

val= input('are you satisfied with detection (y/n)?')
tracker = cv2.TrackerMOSSE_create()
global countFolder
cap = cv2.VideoCapture(cameraNo)
cap.set(3, 640)
cap.set(4, 480)
cap.set(10,cameraBrightness)


count = 0
countSave =0


def saveDataFunc():
    global countFolder
    countFolder = 0
    while os.path.exists( myPath+ str(countFolder)):
        countFolder += 1
    os.makedirs(myPath + str(countFolder))

if saveData:saveDataFunc()


if val is 'y':
    align()
elif val is 'n':
    success, frame = cap.read()
    bbox = cv2.selectROI("Tracking",frame, False)
    tracker.init(frame, bbox)
    
    while True:
        success, img = cap.read()
        success, bbox = tracker.update(img)
        x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
        img_saved=img[x:x+w,y:y+h]
        cv2.imshow('testing',img_saved)
        img_saved = cv2.resize(img_saved,(imgWidth,imgHeight))
        if grayImage:img_saved = cv2.cvtColor(img_saved,cv2.COLOR_BGR2GRAY)
        if saveData:
            blur = cv2.Laplacian(img_saved, cv2.CV_64F).var()
            if count % moduleVal ==0 and blur > minBlur:
                nowTime = time.time()
                cv2.imwrite(myPath + str(countFolder) +
                        '/' + str(countSave)+"_"+ str(int(blur))+"_"+str(nowTime)+".png", img)
                countSave+=1
            count += 1

        if showImage:
            cv2.imshow("Image", img)

        if success:
            drawBox(img,bbox)
        else:
            cv2.putText(img, "Lost", (100, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
        cv2.imshow("Tracking", img)
        if cv2.waitKey(1) & 0xff == ord('q'):
            break